In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/df4.csv')
df2 = pd.read_csv('/content/drive/MyDrive/df4_test.csv')

In [4]:
df2.drop(['uniqueID','date'],axis=1,inplace=True)

In [5]:
df = pd.concat([df1,df2],axis=0)

In [6]:
df['sentiment1'] = df["rating"].apply(lambda x: 1 if x > 5 else 0)

In [7]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,usefulCount,sentiment,clean_review,sentiment1
0,42825,Dicyclomine,Irritable Bowel Syndrome,"""I&#039;ve been on this a month and it&#039;s ...",1,5,negative,month horrible ibs go wave sometimes fine some...,0
1,121198,Nexplanon,Birth Control,"""I got this implanted in my arm, I like it! I ...",9,0,positive,got implanted arm like worry taking pill getti...,1
2,126670,Bupropion / naltrexone,Weight Loss,"""Been taking Contrave for 5 weeks, so just hit...",6,2,neutral,taking contrave week hit max dose pill pill pm...,1
3,113081,Depakote,Migraine Prevention,"""Depakote worked great as a migraine preventat...",1,34,negative,depakote worked great migraine preventative ca...,0
4,150213,Phentermine,Weight Loss,"""I am 23, 5&#039;4 and weighed 181 and current...",10,12,positive,weighed currently weigh started phentermine mg...,1


In [8]:
df =df .dropna()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(2,2),max_features=5000)

X = cv.fit_transform(df['clean_review'].values).toarray()

In [10]:
y = df.sentiment1

In [11]:
X_train = X[0:50000]
X_test = X[50000:]

y_train = y[0:50000]
y_test = y[50000:]

In [12]:

from sklearn.linear_model import LinearRegression
nl_model = LinearRegression().fit(X_train, y_train)

y_pred_lemma_cv=nl_model.predict(X_test)

In [14]:
y_pred_lemma_cv

array([0.56154956, 0.86523714, 0.87582824, ..., 0.30822441, 0.69661218,
       0.75419743])

In [15]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_lemma_cv)


In [16]:
# create a dataframe to store the values for false positive rate, true positive rate and threshold
youdens_table = pd.DataFrame({'TPR': tpr,
                             'FPR': fpr,
                             'Threshold': thresholds})

# calculate the difference between TPR and FPR for each threshold and store the values in a new column 'Difference'
youdens_table['Difference'] = youdens_table.TPR - youdens_table.FPR

# sort the dataframe based on the values of difference 
# 'ascending = False' sorts the data in descending order
# 'reset_index' resets the index of the dataframe
# 'drop = True' drops the previous index
youdens_table = youdens_table.sort_values('Difference', ascending = False).reset_index(drop = True)

# print the first five observations
youdens_table.head()

,TPR,FPR,Threshold,Difference
0,0.703158,0.274794,0.490011,0.428364
1,0.703243,0.274888,0.489993,0.428355
2,0.701465,0.273110,0.492042,0.428355
3,0.701803,0.273484,0.491381,0.428319
4,0.701295,0.273016,0.492146,0.428279


In [18]:
y_pred_youden = [ 0 if x < 0.49 else 1 for x in y_pred_lemma_cv]

In [20]:
from sklearn.metrics import accuracy_score,confusion_matrix
print('accuracy score:',accuracy_score(y_test,y_pred_youden))

accuracy score: 0.7136317169651984


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_lemma_cv))

              precision    recall  f1-score   support

           0       0.69      0.74      0.71     10688
           1       0.75      0.69      0.72     11811

    accuracy                           0.72     22499
   macro avg       0.72      0.72      0.72     22499
weighted avg       0.72      0.72      0.72     22499



In [21]:
from sklearn.metrics import precision_score , recall_score , f1_score
print('precision score:',precision_score(y_test,y_pred_youden))
print('recall score:',recall_score(y_test,y_pred_youden))
print('f1 score:',f1_score(y_test,y_pred_youden))


precision score: 0.738747553816047
recall score: 0.7031580729828126
f1 score: 0.7205135990977312
